## AoC 2024 - Day 16

In [ ]:
#from icecream import ic
import heapq

data = open('test.txt').read().split('\n')

start = None
end = None

# #Same, but using a dictionary, with keys being (row, column) tuples
# #Makes in/out of range checks dead simple, using grid.get((r,c))
# #    None if it doesn't exist, value otherwise
grid = {}
for r, row in enumerate(data):
    for c, char in enumerate(row):
        grid[(r,c)] = char
        if char == "S": start = (r,c)
        if char == "E": end = (r,c)


dir =  ">"
queue = []

# Priority queue with (F-score, node)
heapq.heappush(queue, (0, start, dir))
visited = set()
cell_scores = {start: 0}

while queue:
    _, current, previous_dir = heapq.heappop(queue)
    
    print(f"{current=}")
    if current == end:
        
        cost = 0
        # Reconstruct the path and return
        path = []
        #print(print(came_from))
        while current in visited:
            #print(current)
            cost = max(cost, visited[current][1])
            path.append(current)
            current = visited[current][0]
        path.append(start)
        path.reverse()
        break

    if current in visited: continue

    visited.add((current))
    
    for dr, dc, dir_new in [(0, 1, ">"), (0, -1, "<"), (1, 0, "v"), (-1, 0, "^")]:
        r, c = current[0] + dr, current[1] + dc

        neighbor = (r, c)

        if grid.get(neighbor) == "#": continue


        move_cost = cell_scores[current] + 1
        if previous_dir != dir_new:
            move_cost += 1000

        if neighbor not in cell_scores or move_cost < cell_scores[neighbor]:
                dir = dir_new
                cell_scores[neighbor] = move_cost

                heapq.heappush(queue, (move_cost, neighbor, dir))
                #visited[neighbor] = (current, move_cost)

    print(f"{queue=}")
    print(f"{cell_scores=}")
    print("")



print(f"{grid[start]} at {start}, {grid[end]} at {end}")

print(f"steps: {len(path)}")
print(f"Shortest path:", path, cost)


In [ ]:
import heapq

class Node:
    def __init__(self, x, y, direction, move_cost=1, turn_cost=1000):
        self.x = x
        self.y = y
        self.direction = direction  # 'N', 'E', 'S', or 'W'
        self.move_cost = move_cost
        self.turn_cost = turn_cost
        self.visited = False
        self.distance = float('inf')
        self.prev = None
        
    def __lt__(self, other):
        return self.distance < other.distance
        
    def __eq__(self, other):
        return (self.x, self.y, self.direction) == (other.x, other.y, other.direction)

def get_neighbors(node, maze):
    directions = {'N': (-1, 0), 'E': (0, 1), 'S': (1, 0), 'W': (0, -1)}
    neighbors = []
    
    for dir, (dx, dy) in directions.items():
        nx, ny = node.x + dx, node.y + dy
        
        # Check if the new position is within maze bounds and not a wall
        if 0 <= nx < len(maze) and 0 <= ny < len(maze[0]) and maze[nx][ny] != '#':
            turn_cost = 0 if node.direction == dir else node.turn_cost
            neighbors.append((nx, ny, dir, node.move_cost + turn_cost))
    
    return neighbors

def dijkstra(maze, start, end):
    start_x, start_y = start
    end_x, end_y = end
    nodes = {}

    # Initialize nodes (four per cell for each direction)
    for x in range(len(maze)):
        for y in range(len(maze[0])):
            if maze[x][y] != '#':
                for direction in 'NESW':
                    nodes[(x, y, direction)] = Node(x, y, direction)
    
    # Define start node
    start_node = nodes[(start_x, start_y, 'E')]
    start_node.distance = 0
    
    # Priority queue for nodes to visit; initialized with start node
    queue = [(start_node.distance, start_node)]
    heapq.heapify(queue)

    shortest_path_distance = None
    
    while queue:
        current_distance, current_node = heapq.heappop(queue)
        
        if (current_node.x, current_node.y) == (end_x, end_y):
            path = []
            current = current_node
            while current != start_node:
                path.append((current.x, current.y, current.direction))
                current = current.prev
            path.append((start_node.x, start_node.y, start_node.direction))
            path.reverse()
            return path
            # If end is reached, construct the path
            if shortest_path_distance is None:
                shortest_path_distance = current_node.distance
                print(shortest_path_distance)
            return construct_path(nodes, start_node, current_node), [(node.x, node.y, node.distance) for node in nodes.values() if node.distance <= shortest_path_distance]
        
        if current_node.visited:
            continue
        
        current_node.visited = True
        
        for nx, ny, ndir, cost in get_neighbors(current_node, maze):
            neighbor = nodes[(nx, ny, ndir)]
            if not neighbor.visited and current_node.distance + cost < neighbor.distance:
                
                neighbor.distance = current_node.distance + cost
                neighbor.prev = current_node
                heapq.heappush(queue, (neighbor.distance, neighbor))
    

    return "No path found"

def construct_path(nodes, start_node, end_node):
    path = []
    current = end_node
    while current != start_node:
        path.append((current.x, current.y, current.direction))
        current = current.prev
    path.append((start_node.x, start_node.y, start_node.direction))
    path.reverse()
    return path

# Maze representation where 'S' marks the start, 'E' marks the end, '.' marks open paths, and '#' marks walls
maze = open('test.txt').read().split('\n')


start = None
end = None
for r, row in enumerate(maze):
    for c, char in enumerate(row):
        grid[(r,c)] = char
        if char == "S": start = (r,c)
        if char == "E": end = (r,c)


#print(f"Start: {start}, End: {end}")
path = dijkstra(maze, start, end)


# Print the path
facing = "E"
cost = -1
steps = -1
turns = 0
for step in path:

    if step[2] != facing:
        turns += 1
        cost += 1000
        facing = step[2]
    
    cost += 1
    steps += 1
    #print(f"Cell: ({step[0]:2}, {step[1]:2}), Facing: {step[2]}, cost: {cost}")
print(f"steps: {steps}, Turns: {turns}, Cost: {cost}")
#print(f"Cells: {len(cells)}")



In [ ]:
import heapq

class Node:
    def __init__(self, x, y, direction):
        self.x = x
        self.y = y
        self.direction = direction
        self.distance = float('inf')
        self.prev = None
        self.processed = False

    def __lt__(self, other):
        return self.distance < other.distance

def get_neighbors(maze, x, y, direction, move_cost=1, turn_cost=1000):
    neighbors = []
    directions = {'N': ('W', 'E'), 'S': ('E', 'W'), 'E': ('N', 'S'), 'W': ('S', 'N')}
    dx, dy = {'N': (-1, 0), 'E': (0, 1), 'S': (1, 0), 'W': (0, -1)}[direction]

    # Move forward if no wall is encountered
    if 0 <= x + dx < len(maze) and 0 <= y + dy < len(maze[0]) and maze[x + dx][y + dy] != '#':
        neighbors.append((x + dx, y + dy, direction, move_cost))

    # Check turning left and right from the current direction
    for turn_dir in directions[direction]:
        neighbors.append((x, y, turn_dir, turn_cost))

    return neighbors

def dijkstra(maze, start, end):
    move_cost = 1
    turn_cost = 1000
    nodes = {(x, y, direction): Node(x, y, direction) for x in range(len(maze)) for y in range(len(maze[0])) for direction in 'NESW' if maze[x][y] != '#'}

    start_x, start_y, end_x, end_y = *start, *end
    start_node = nodes[(start_x, start_y, 'E')]
    start_node.distance = 0

    queue = [(0, start_node)]
    shortest_path_cost = None
    visited_cells = set()

    while queue:
        current_distance, current_node = heapq.heappop(queue)
        current_node.processed = True

        if (current_node.x, current_node.y) == (end_x, end_y):
            if shortest_path_cost is None or current_distance < shortest_path_cost:
                shortest_path_cost = current_distance

        if shortest_path_cost is not None and current_distance > shortest_path_cost:
            break

        for nx, ny, ndir, cost in get_neighbors(maze, current_node.x, current_node.y, current_node.direction, move_cost, turn_cost):
            neighbor = nodes[(nx, ny, ndir)]
            new_distance = current_distance + cost
            if new_distance < neighbor.distance:
                neighbor.distance = new_distance
                neighbor.prev = current_node
                heapq.heappush(queue, (new_distance, neighbor))
        visited_cells.add((current_node.x, current_node.y))

    return shortest_path_cost, len(visited_cells)

# Define maze and start/end coordinates as before
maze = open('test.txt').read().split('\n')


start = None
end = None
for r, row in enumerate(maze):
    for c, char in enumerate(row):
        grid[(r,c)] = char
        if char == "S": start = (r,c)
        if char == "E": end = (r,c)

# Use the dijkstra function to get the cost of the shortest paths and total number of cells on those paths
shortest_path_cost, total_visited_cells = dijkstra(maze, start, end)
print(f"The cost of the shortest path is: {shortest_path_cost}")
print(f"The total number of cells on the shortest paths is: {total_visited_cells}")